In [2]:
import os

In [3]:
# + SubVert

In [4]:
os.environ["CUDA_VISIBLE_DEVICES"] = "6"

In [5]:
import tensorflow as tf
import keras.backend.tensorflow_backend as KTF
 
config = tf.ConfigProto()  
config.gpu_options.allow_growth=True  
session = tf.Session(config=config)
 
KTF.set_session(session)

Using TensorFlow backend.


In [6]:
import csv
import datetime
import time
import json
import itertools
import random
import os
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt

In [7]:
import nltk 
from nltk.tokenize import word_tokenize
from sklearn.metrics import roc_auc_score, average_precision_score, precision_recall_fscore_support

In [8]:
import numpy
import keras
from keras.utils.np_utils import *
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

from keras.layers import Embedding, concatenate
from keras.layers import Dense, Input, Flatten, average,Lambda

from keras.layers import *
from keras.models import Model, load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint

from keras import backend as K
from keras.engine.topology import Layer, InputSpec
from keras import initializers #keras2
from keras.utils import plot_model
import numpy as np
from sklearn.metrics import accuracy_score, classification_report
from keras.optimizers import *

In [9]:
import os

In [10]:
from datetime import datetime
import time
def trans2tsp(timestr):
    return int(time.mktime(datetime.strptime(timestr, '%m/%d/%Y %I:%M:%S %p').timetuple()))

In [11]:
def newsample(nnn,ratio):
    if ratio >len(nnn):
        return random.sample(nnn*(ratio//len(nnn)+1),ratio)
    else:
        return random.sample(nnn,ratio)

In [12]:
def shuffle(pn,labeler,pos):
    index=np.arange(pn.shape[0])
    pn=pn[index]
    labeler=labeler[index]
    pos=pos[index]
    
    for i in range(pn.shape[0]):
        index=np.arange(npratio+1)
        pn[i,:]=pn[i,index]
        labeler[i,:]=labeler[i,index]
    return pn,labeler,pos

In [13]:
def dcg_score(y_true, y_score, k=10):
    order = np.argsort(y_score)[::-1]
    y_true = np.take(y_true, order[:k])
    gains = 2 ** y_true - 1
    discounts = np.log2(np.arange(len(y_true)) + 2)
    return np.sum(gains / discounts)


def ndcg_score(y_true, y_score, k=10):
    best = dcg_score(y_true, y_true, k)
    actual = dcg_score(y_true, y_score, k)
    return actual / best


def mrr_score(y_true, y_score):
    order = np.argsort(y_score)[::-1]
    y_true = np.take(y_true, order)
    rr_score = y_true / (np.arange(len(y_true)) + 1)
    return np.sum(rr_score) / np.sum(y_true)

In [14]:
#REad News

In [15]:
def read_news(path,filenames):
    news={}
    category=[]
    subcategory=[]
    news_index={}
    index=1
    word_dict={}
    word_index=1
    with open(os.path.join(path,filenames)) as f:
        lines=f.readlines()
    for line in lines:
        splited = line.strip('\n').split('\t')
        doc_id,vert,subvert,title= splited[0:4]
        news_index[doc_id]=index
        index+=1
        category.append(vert)
        subcategory.append(subvert)
        title = title.lower()
        title=word_tokenize(title)
        news[doc_id]=[vert,subvert,title]
        for word in title:
            word = word.lower()
            if not(word in word_dict):
                word_dict[word]=word_index
                word_index+=1
    category=list(set(category))
    subcategory=list(set(subcategory))
    category_dict={}
    index=1
    for c in category:
        category_dict[c]=index
        index+=1
    subcategory_dict={}
    index=1
    for c in subcategory:
        subcategory_dict[c]=index
        index+=1
    return news,news_index,category_dict,subcategory_dict,word_dict

In [16]:
data_root_path = '/data/qit16/MIND-Full/'
embedding_path = '/data/qit16/word_embedding/'

In [17]:
%time news,news_index,category_dict,subcategory_dict,word_dict = read_news(data_root_path,'docs.tsv')

CPU times: user 25.8 s, sys: 880 ms, total: 26.7 s
Wall time: 26.7 s


In [18]:
MAX_SENTENCE = 30

In [19]:
def get_doc_input(news,news_index,category,subcategory,word_dict):
    news_num=len(news)+1
    news_title=np.zeros((news_num,MAX_SENTENCE),dtype='int32')
    news_vert=np.zeros((news_num,),dtype='int32')
    news_subvert=np.zeros((news_num,),dtype='int32')
    for key in news:    
        vert,subvert,title=news[key]
        doc_index=news_index[key]
        news_vert[doc_index]=category[vert]
        news_subvert[doc_index]=subcategory[subvert]
        for word_id in range(min(MAX_SENTENCE,len(title))):
            news_title[doc_index,word_id]=word_dict[title[word_id].lower()]
        
    return news_title,news_vert,news_subvert

In [20]:
news_title,news_vert,news_subvert=get_doc_input(news,news_index,category_dict,subcategory_dict,word_dict)

In [21]:
def load_matrix(embedding_path,word_dict):
    embedding_matrix = np.zeros((len(word_dict)+1,300))
    have_word=[]
    with open(os.path.join(embedding_path,'glove.840B.300d.txt'),'rb') as f:
        while True:
            l=f.readline()
            if len(l)==0:
                break
            l=l.split()
            word = l[0].decode()
            if word in word_dict:
                index = word_dict[word]
                tp = [float(x) for x in l[1:]]
                embedding_matrix[index]=np.array(tp)
                have_word.append(word)
    return embedding_matrix,have_word

In [22]:
#End Read News

In [23]:
# Read Session

In [24]:
def read_clickhistory(path,filename):
    
    lines = []
    userids = []
    with open(os.path.join(data_root_path,filename)) as f:
        lines = f.readlines()
        
    sessions = []
    for i in range(len(lines)):
        if 'Full' in data_root_path:
            _,click, imp = lines[i].strip().split('\t')
        else:
            _,_,click, imp = lines[i].strip().split('\t')
        clikcs = click.split('#N#')
        true_click = []
        for click in clikcs:
            t = click.split('#TAB#')[0]
            if t =='':
                continue
            true_click.append(t)
        pos, neg, _ = imp.split('#TAB#')
        pos = pos.split()
        neg = neg.split()
        sessions.append([true_click,pos,neg])
    return sessions

In [25]:
train_session = read_clickhistory(data_root_path,'train_sam2.tsv')

In [26]:
def read_part_clickhistory(path,filename,num=200000):
    
    lines = []
    userids = []
    with open(os.path.join(data_root_path,filename)) as f:
        for i in range(num):
            l = f.readline()
            lines.append(l)
    print(num)
        
    sessions = []
    for i in range(len(lines)):
        if 'Full' in data_root_path:
            _,click, imp = lines[i].strip().split('\t')
        else:
            _,_,click, imp = lines[i].strip().split('\t')
        clikcs = click.split('#N#')
        true_click = []
        for click in clikcs:
            t = click.split('#TAB#')[0]
            if t =='':
                continue
            true_click.append(t)
        pos, neg, _ = imp.split('#TAB#')
        pos = pos.split()
        neg = neg.split()
        sessions.append([true_click,pos,neg])
    return sessions

In [27]:
test_session = read_part_clickhistory(data_root_path,'test.tsv')

200000


In [28]:
# val_session = read_clickhistory(data_root_path,'val_sam2.tsv')

In [29]:
#Parser User

In [30]:
MAX_ALL = 50

In [31]:
def parse_user(session):
    user_num = len(session)
    user={'click': np.zeros((user_num,MAX_ALL),dtype='int32'),}
    for user_id in range(len(session)):
        tclick = []
        click, pos, neg =session[user_id]
        for i in range(len(click)):
            tclick.append(news_index[click[i]])
        click = tclick

        if len(click) >MAX_ALL:
            click = click[-MAX_ALL:]
        else:
            click=[0]*(MAX_ALL-len(click)) + click
            
        user['click'][user_id] = np.array(click)
    return user

In [32]:
train_user = parse_user(train_session)

In [33]:
test_user = parse_user(test_session)

In [34]:
# val_user = parse_user(val_session)

In [35]:
#Get Training Data

In [36]:
npratio=4

In [37]:
def get_train_input(session):
    sess_pos = []
    sess_neg = []
    user_id = []
    for sess_id in range(len(session)):
        sess = session[sess_id]
        _, poss, negs=sess
        for i in range(len(poss)):
            pos = poss[i]
            neg=newsample(negs,npratio)
            sess_pos.append(pos)
            sess_neg.append(neg)
            user_id.append(sess_id)
    print(len(user_id))
    sess_all = np.zeros((len(sess_pos),1+npratio),dtype='int32')
    label = np.zeros((len(sess_pos),1+npratio))
    for sess_id in range(sess_all.shape[0]):
        pos = sess_pos[sess_id]
        negs = sess_neg[sess_id]
        sess_all[sess_id,0] = news_index[pos]
        index = 1
        for neg in negs:
            sess_all[sess_id,index] = news_index[neg]
            index+=1
        #index = np.random.randint(1+npratio)
        label[sess_id,0]=1
    user_id = np.array(user_id, dtype='int32')
    
    return sess_all, user_id, label

In [38]:
train_sess, train_user_id, train_label = get_train_input(train_session)

3383654


In [39]:
def get_test_input(session):
    
    Impressions = []
    userid = []
    for sess_id in range(len(session)):
        _, poss, negs = session[sess_id]
        imp = {'labels':[],
                'docs':[]}
        userid.append(sess_id)
        for i in range(len(poss)):
            docid = news_index[poss[i]]
            imp['docs'].append(docid)
            imp['labels'].append(1)
        for i in range(len(negs)):
            docid = news_index[negs[i]]
            imp['docs'].append(docid)
            imp['labels'].append(0)
        Impressions.append(imp)
        
    userid = np.array(userid,dtype='int32')
    
    return Impressions, userid,

In [40]:
test_impressions, test_userids = get_test_input(test_session)

In [41]:
KG_root_path = '/data/qit16/Opendata_KG/'

In [42]:
with open(os.path.join(KG_root_path,'V21UrlDocs22_all_josn.tsv')) as f:
    lines = f.readlines()
    
entity_dict = {}
news_entity = np.zeros((len(news_index)+1,5),dtype='int32')
entity_index = 1

for i in range(len(lines)):
    data = json.loads(lines[i].strip('\n'))
    nix = news_index[data['doc_id']]
    ct = 0
    for e in data['entities']:
        eid = e['WikidataId']
        if not eid in entity_dict:
            entity_dict[eid] = entity_index
            entity_index += 1
            
        news_entity[nix,ct] = entity_dict[eid]
        ct += 1
        if ct == 5:
            break

In [43]:
# with open(os.path.join(KG_root_path,'description.txt')) as f:
#     lines = f.readlines()
    
# entity_desc = {}
# for i in range(len(lines)):
#     eid,text = lines[i].strip('\n').split('\t')
#     if not eid in entity_dict:
#         continue
#     entity_desc[eid] = text
    
# for nid in entity_dict:
#     if not nid in entity_desc:
#         entity_desc[nid] = ''
        
# with open('entity_desc.json','w') as f:
#     json.dump(entity_desc,f)

In [44]:
with open('entity_desc.json','r') as f:
    entity_desc_dict = json.load(f)

In [45]:
len(word_dict)

64963

In [46]:
# word_dict = {}
word_index = len(word_dict)+1

entity_desc = np.zeros((len(entity_dict)+1,10),dtype='int32')

for eid in entity_desc_dict:
    text = entity_desc_dict[eid]
    text = word_tokenize(text.lower())
    for i in range(min(len(text),10)):
        word = text[i]
        if not word in word_dict:
            word_dict[word] = word_index
            word_index += 1
        entity_desc[entity_dict[eid],i] = word_dict[word]

In [47]:
len(word_dict)

70819

In [48]:
title_word_embedding_matrix, have_word = load_matrix(embedding_path,word_dict)

In [49]:
# entity_word_embedding_matrix, have_word = load_matrix(embedding_path,entity_word_dict)

In [50]:
class NewsFetcher():
    def __init__(self,news_title,news_entity,entity_desc):
        self.news_title = news_title
        self.news_entity = news_entity
        self.entity_desc = entity_desc
        
    def fetch(self,docids):
        bz,n = docids.shape
        news_title = self.news_title[docids] #(N,30)
        news_entity_ids = self.news_entity[docids]
        news_entity_texts = self.entity_desc[news_entity_ids] #(N,30,10)
        news_entity_texts = news_entity_texts.reshape((bz,n,-1))
        
        news_info = np.concatenate([news_title,news_entity_texts],axis=-1)
        
        return news_info
    
    def fetch_dim1(self,docids):
        bz, = docids.shape
        news_title = self.news_title[docids] #(N,30)
        news_entity_ids = self.news_entity[docids]
        news_entity_texts = self.entity_desc[news_entity_ids] #(N,30,10)
        news_entity_texts = news_entity_texts.reshape((bz,-1))
        
        news_info = np.concatenate([news_title,news_entity_texts],axis=-1)
        
        return news_info

In [51]:
news_fetcher = NewsFetcher(news_title,news_entity,entity_desc)

In [52]:
from keras.utils import Sequence

class get_hir_train_generator(Sequence):
    def __init__(self,news_fetcher,clicked_news,user_id, news_id, label, batch_size):
        self.news_fetcher = news_fetcher
        self.clicked_news = clicked_news

        self.user_id = user_id
        self.doc_id = news_id
        self.label = label
        
        self.batch_size = batch_size
        self.ImpNum = self.label.shape[0]
        
    def __len__(self):
        return int(np.ceil(self.ImpNum / float(self.batch_size)))
    

    def __getitem__(self, idx):
        start = idx*self.batch_size
        ed = (idx+1)*self.batch_size
        if ed> self.ImpNum:
            ed = self.ImpNum
        label = self.label[start:ed]
        
        doc_ids = self.doc_id[start:ed]
        info= self.news_fetcher.fetch(doc_ids)
        
        user_ids = self.user_id[start:ed]
        clicked_ids = self.clicked_news[user_ids]
        user_info = self.news_fetcher.fetch(clicked_ids)
        
        click_mask = clicked_ids>0
        click_mask = np.array(click_mask,dtype='float32')
        

        return ([info, user_info],[label])

In [53]:
train_generator = get_hir_train_generator(news_fetcher,train_user['click'],train_user_id,train_sess,train_label,32)

In [54]:
class get_hir_user_generator(Sequence):
    def __init__(self,news_emb,clicked_news,batch_size):
        self.news_emb = news_emb

        self.clicked_news = clicked_news

        self.batch_size = batch_size
        self.ImpNum = self.clicked_news.shape[0]
        
    def __len__(self):
        return int(np.ceil(self.ImpNum / float(self.batch_size)))

    
    def __get_news(self,docids):
        news_emb = self.news_emb[docids]

        return news_emb
    
    
    def __getitem__(self, idx):
        start = idx*self.batch_size
        ed = (idx+1)*self.batch_size
        if ed> self.ImpNum:
            ed = self.ImpNum
            
        clicked_ids = self.clicked_news[start:ed]
        user_title = self.__get_news(clicked_ids)

        
        return user_title

In [55]:
class get_hir_news_generator(Sequence):
    def __init__(self,news_fetcher,batch_size):
        self.news_fetcher = news_fetcher

        self.batch_size = batch_size
        self.ImpNum = len(news_index)+1
        
    def __len__(self):
        return int(np.ceil(self.ImpNum / float(self.batch_size)))

    
    def __get_news(self,docids):
        news_emb = self.news_emb[docids]

        return news_emb
    
    
    def __getitem__(self, idx):
        start = idx*self.batch_size
        ed = (idx+1)*self.batch_size
        if ed> self.ImpNum:
            ed = self.ImpNum
            
        docids = np.array([i for i in range(start,ed)])
            
        info = self.news_fetcher.fetch_dim1(docids)

        return info

In [56]:
news_generator = get_hir_news_generator(news_fetcher,32)

In [57]:
def evaluate(test_impressions,user_scoring):
    AUC = []
    MRR = []
    nDCG5 = []
    nDCG10 = []
    for i in range(len(test_impressions)):
        labels = test_impressions[i]['labels']
        nids = test_impressions[i]['docs']

        uv = user_scoring[i]

        nvs = news_scoring[nids]
        score = np.dot(nvs,uv)

        auc = roc_auc_score(labels,score)
        mrr = mrr_score(labels,score)
        ndcg5 = ndcg_score(labels,score,k=5)
        ndcg10 = ndcg_score(labels,score,k=10)
    
        AUC.append(auc)
        MRR.append(mrr)
        nDCG5.append(ndcg5)
        nDCG10.append(ndcg10)
        
    AUC = np.array(AUC).mean()
    MRR = np.array(MRR).mean()
    nDCG5 = np.array(nDCG5).mean()
    nDCG10 = np.array(nDCG10).mean()

    
    return AUC, MRR, nDCG5, nDCG10

In [58]:
class Attention(Layer):
 
    def __init__(self, nb_head, size_per_head, **kwargs):
        self.nb_head = nb_head
        self.size_per_head = size_per_head
        self.output_dim = nb_head*size_per_head
        super(Attention, self).__init__(**kwargs)
 
    def build(self, input_shape):
        self.WQ = self.add_weight(name='WQ',
                                  shape=(input_shape[0][-1], self.output_dim),
                                  initializer='glorot_uniform',
                                  trainable=True)
        self.WK = self.add_weight(name='WK',
                                  shape=(input_shape[1][-1], self.output_dim),
                                  initializer='glorot_uniform',
                                  trainable=True)
        self.WV = self.add_weight(name='WV',
                                  shape=(input_shape[2][-1], self.output_dim),
                                  initializer='glorot_uniform',
                                  trainable=True)
        super(Attention, self).build(input_shape)
 
    def Mask(self, inputs, seq_len, mode='mul'):
        if seq_len == None:
            return inputs
        else:
            mask = K.one_hot(seq_len[:,0], K.shape(inputs)[1])
            mask = 1 - K.cumsum(mask, 1)
            for _ in range(len(inputs.shape)-2):
                mask = K.expand_dims(mask, 2)
            if mode == 'mul':
                return inputs * mask
            if mode == 'add':
                return inputs - (1 - mask) * 1e12
 
    def call(self, x):
        #如果只传入Q_seq,K_seq,V_seq，那么就不做Mask
        #如果同时传入Q_seq,K_seq,V_seq,Q_len,V_len，那么对多余部分做Mask
        if len(x) == 3:
            Q_seq,K_seq,V_seq = x
            Q_len,V_len = None,None
        elif len(x) == 5:
            Q_seq,K_seq,V_seq,Q_len,V_len = x
        #对Q、K、V做线性变换
        seq_length = 10
        
        Q_seq = K.dot(Q_seq, self.WQ)
        Q_seq = K.reshape(Q_seq, (-1, K.shape(Q_seq)[1], self.nb_head, self.size_per_head))
        Q_seq = K.permute_dimensions(Q_seq, (0,2,1,3))

        K_seq = K.dot(K_seq, self.WK)
        K_seq = K.reshape(K_seq, (-1, K.shape(K_seq)[1], self.nb_head, self.size_per_head))
        K_seq = K.permute_dimensions(K_seq, (0,2,1,3))
        V_seq = K.dot(V_seq, self.WV)
        V_seq = K.reshape(V_seq, (-1, K.shape(V_seq)[1], self.nb_head, self.size_per_head))
        V_seq = K.permute_dimensions(V_seq, (0,2,1,3))
        #计算内积，然后mask，然后softmax
        A = K.batch_dot(Q_seq, K_seq, axes=[3,3]) / self.size_per_head**0.5
        A = K.permute_dimensions(A, (0,3,2,1))
        A = self.Mask(A, V_len, 'add')
        A = K.permute_dimensions(A, (0,3,2,1))
        A = K.softmax(A)
        #输出并mask
        O_seq = K.batch_dot(A, V_seq, axes=[3,2])
        O_seq = K.permute_dimensions(O_seq, (0,2,1,3))
        O_seq = K.reshape(O_seq, (-1, K.shape(O_seq)[1], self.output_dim))
        O_seq = self.Mask(O_seq, Q_len, 'mul')
        return O_seq
 
    def compute_output_shape(self, input_shape):
        return (input_shape[0][0], input_shape[0][1], self.output_dim)

In [59]:
MAX_SENT_LENGTH=30
MAX_SENTS=50

In [60]:
def AttentivePooling(dim1,dim2):
    vecs_input = Input(shape=(dim1,dim2),dtype='float32') # 50, 400
    user_vecs =Dropout(0.2)(vecs_input)
    user_att = Dense(200,activation='tanh')(user_vecs) # 50, 200
    user_att = keras.layers.Flatten()(Dense(1)(user_att)) # 50,
    user_att = Activation('softmax')(user_att)
    user_vec = keras.layers.Dot((1,1))([user_vecs,user_att])
    model = Model(vecs_input,user_vec)
    return model

In [61]:
def AttentivePoolingAMask(dim1,dim2):
    vecs_input = Input(shape=(dim1,dim2),dtype='float32')
    mask_input = Input(shape=(dim1,),dtype='float32')

    user_vecs =Dropout(0.2)(vecs_input)
    user_att = Dense(200,activation='tanh')(user_vecs)
    user_att = keras.layers.Flatten()(Dense(1)(user_att))
    user_att = keras.layers.Lambda(lambda x: x[0]-100*(1-x[1]))([user_att,mask_input])
    user_att = Activation('softmax')(user_att)
    user_vec = keras.layers.Dot((1,1))([user_vecs,user_att])
    model = Model([vecs_input,mask_input],user_vec)
    return model

In [62]:
def get_text_encoder(mode,length,word_embedding_layer):

    sentence_input = Input(shape=(length,),dtype='int32')
    
    #word_embedding_layer = Embedding(word_embedding_matrix.shape[0], 300, weights=[word_embedding_matrix],trainable=True)
    word_vecs = word_embedding_layer(sentence_input)
    droped_vecs = Dropout(0.2)(word_vecs)
    
    if 'SelfAtt' in mode:
        word_rep = Attention(20,20)([droped_vecs]*3)
    elif 'CNN' in mode:
        word_rep = Conv1D(400,3,padding='same')(droped_vecs)
    
    if 'ReLU' in mode:
        print('use relu')
        word_rep = Activation('relu')(word_rep)
    
    droped_rep = Dropout(0.2)(word_rep)    
    title_vec = AttentivePooling(length,400)(droped_rep)

    sentEncodert = Model(sentence_input, title_vec)
    return sentEncodert

In [63]:
def get_news_encoder(mode):
    # shape=(50,30+5*10,)
    news_input = Input(shape=(30+5*10,),dtype='int32')
    
    title_input = Lambda(lambda x:x[:,:30])(news_input)
    entity_input = Lambda(lambda x:x[:,30:])(news_input) #(bz,5*10)
    entity_input = Reshape((5,10))(entity_input)

    word_embedding_layer = Embedding(title_word_embedding_matrix.shape[0], 300, weights=[title_word_embedding_matrix],trainable=True)
    
    title_encoder = get_text_encoder(mode,30,word_embedding_layer)

    vec = title_encoder(title_input)


    sentEncodert = Model(news_input, vec)
    return sentEncodert

In [75]:
def get_user_encoder(mode,user_relu):
    user_vecs_input = Input(shape=(50,400))    
    click_mask_input = Input(shape=(50,))
    
    user_vecs = Dropout(0.2)(user_vecs_input)

    if mode=='NRMS':
        user_vecs = Attention(20,20)([user_vecs]*3)
        user_vec = AttentivePooling(50,400)(user_vecs)
    elif mode in ['NAML',]:
        user_vec = AttentivePooling(50,400)(user_vecs)     
    elif mode in ['NAML-Dense']:
        user_vec = AttentivePooling(50,400)(user_vecs)     
        user_vec = Dense(400)(user_vec)
    if user_relu:
        user_vec = Activation('relu')(user_vec)
        
    return Model(user_vecs_input,user_vec)

In [76]:
DIM_TITLE = 768
def create_model(mode1,mode2,user_relu):
    
    MAX_LENGTH = 30    
    
    news_encoder = get_news_encoder(mode1)
    user_encoder = get_user_encoder(mode2,user_relu)
    
    clicked_title_input = Input(shape=(50,30+5*10,), dtype='int32')    
    title_inputs = Input(shape=(1+npratio,30+5*10,),dtype='int32') 

    user_vecs = TimeDistributed(news_encoder)(clicked_title_input)
    user_vec = user_encoder(user_vecs)
    
    news_vecs = TimeDistributed(news_encoder)(title_inputs)
    news_vecs = Dropout(0.2)(news_vecs)
    
    scores = keras.layers.Dot(axes=-1)([news_vecs,user_vec])
    
    logits = keras.layers.Activation(keras.activations.softmax,name = 'recommend')(scores)     

    model = Model([title_inputs, clicked_title_input,],logits) # max prob_click_positive
    model.compile(loss=['categorical_crossentropy'],
                  optimizer=Adam(lr=0.0001), 
                  #optimizer= SGD(lr=0.01),
                  metrics=['acc'])

    return model,news_encoder,user_encoder

In [ ]:
Res = []
news_encoder_mode = 'SelfAtt-ReLU'
user_encoder_mode = 'NRMS'

model,news_encoder,user_encoder, = create_model(news_encoder_mode,user_encoder_mode)
# model.fit_generator(train_generator,epochs=1)

model.fit_generator(train_generator,epochs=2)
news_scoring = news_encoder.predict_generator(news_generator,verbose=1)
test_user_generator = get_hir_user_generator(news_scoring,test_user['click'][:210000],32)
test_user_scoring = user_encoder.predict_generator(test_user_generator,verbose=1)
g2 = evaluate(test_impressions,test_user_scoring)
Res.append(g2)
    
print(g2,Res)